In [1]:
# ruff: noqa: F401

In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
import time
from datetime import datetime
from typing import Any, cast

import hvplot as hv
import hvplot.polars
import numpy as np
import polars as pl

from encomp.fluids import Fluid, Water
from encomp.misc import isinstance_types
from encomp.units import Quantity as Q
from encomp.units import Unit
from encomp.utypes import (
    DT,
    MT,
    Currency,
    Density,
    Dimensionality,
    Dimensionless,
    Length,
    Mass,
    Numpy1DArray,
    Power,
    Pressure,
    Temperature,
    TemperatureDifference,
    Time,
    UnitsContainer,
    UnknownDimensionality,
)

In [4]:
# Properties to compare
attrs = ("D", "H", "S", "V", "M")

# Grid resolution for heatmaps
N_grid = 50  # Reduced for performance, increase for higher resolution

In [5]:
# ===== Heatmap 1: Temperature vs Pressure =====
print("Calculating T vs P grid...")

# Create 2D grid
T_range = np.linspace(0, 700, N_grid)  # 0 to 700 degC
P_range = np.linspace(0.1, 200, N_grid)  # 0.1 to 200 bar (avoid zero pressure)
T_grid, P_grid = np.meshgrid(T_range, P_range)

# Flatten for calculation
T_flat = cast(Q[Temperature, Numpy1DArray], Q(T_grid.flatten(), "degC"))
P_flat = cast(Q[Pressure, Numpy1DArray], Q(P_grid.flatten(), "bar"))

# Calculate properties for both formulations
w97_tp = Fluid("IF97::Water", T=T_flat, P=P_flat)
w95_tp = Fluid("Water", T=T_flat, P=P_flat)

# Create dataframe
df_tp = pl.DataFrame(
    {
        "T": T_flat.m,
        "P": P_flat.m,
    }
    | {f"w95_{a}": getattr(w95_tp, a).m for a in attrs}
    | {f"w97_{a}": getattr(w97_tp, a).m for a in attrs}
)

# Create heatmaps for each property
plots_tp = [
    df_tp.with_columns((pl.col(f"w95_{a}") / pl.col(f"w97_{a}")).alias(f"ratio_{a}")).hvplot.heatmap(
        x="T",
        y="P",
        C=f"ratio_{a}",
        title=f"{a}: IAPWS-95/97 ratio (T vs P)",
        xlabel="Temperature (°C)",
        ylabel="Pressure (bar)",
        cmap="RdBu_r",
        clim=(0.95, 1.05),  # Focus on ±5% difference
        colorbar=True,
        width=500,
        height=400,
    )
    for a in attrs
]

print("T vs P plots ready")
sum(plots_tp[1:], plots_tp[0]).cols(2)

Calculating T vs P grid...
T vs P plots ready


:Layout
   .HeatMap.I   :HeatMap   [T,P]   (ratio_D)
   .HeatMap.II  :HeatMap   [T,P]   (ratio_H)
   .HeatMap.III :HeatMap   [T,P]   (ratio_S)
   .HeatMap.IV  :HeatMap   [T,P]   (ratio_V)
   .HeatMap.V   :HeatMap   [T,P]   (ratio_M)

In [6]:
# ===== Heatmap 2: Temperature vs Quality =====
print("Calculating T vs Q grid...")

# Create 2D grid
T_range = np.linspace(0, 350, N_grid)  # 0 to 350 degC (within two-phase region limits)
Q_range = np.linspace(0, 1, N_grid)  # Quality from 0 to 1
T_grid, Q_grid = np.meshgrid(T_range, Q_range)

# Flatten for calculation
T_flat = cast(Q[Temperature, Numpy1DArray], Q(T_grid.flatten(), "degC"))
Q_flat = cast(Q[Dimensionless, Numpy1DArray], Q(Q_grid.flatten(), "dimensionless"))

# Calculate properties for both formulations
w97_tq = Fluid("IF97::Water", T=T_flat, Q=Q_flat)
w95_tq = Fluid("Water", T=T_flat, Q=Q_flat)

# Create dataframe
df_tq = pl.DataFrame(
    {
        "T": T_flat.m,
        "Q": Q_flat.m,
    }
    | {f"w95_{a}": getattr(w95_tq, a).m for a in attrs}
    | {f"w97_{a}": getattr(w97_tq, a).m for a in attrs}
)

# Create heatmaps for each property
plots_tq = [
    df_tq.with_columns((pl.col(f"w95_{a}") / pl.col(f"w97_{a}")).alias(f"ratio_{a}")).hvplot.heatmap(
        x="T",
        y="Q",
        C=f"ratio_{a}",
        title=f"{a}: IAPWS-95/97 ratio (T vs Q)",
        xlabel="Temperature (°C)",
        ylabel="Quality (vapor fraction)",
        cmap="RdBu_r",
        clim=(0.95, 1.05),
        colorbar=True,
        width=500,
        height=400,
    )
    for a in attrs
]

print("T vs Q plots ready")
sum(plots_tq[1:], plots_tq[0]).cols(2)

Calculating T vs Q grid...
T vs Q plots ready


:Layout
   .HeatMap.I   :HeatMap   [T,Q]   (ratio_D)
   .HeatMap.II  :HeatMap   [T,Q]   (ratio_H)
   .HeatMap.III :HeatMap   [T,Q]   (ratio_S)
   .HeatMap.IV  :HeatMap   [T,Q]   (ratio_V)
   .HeatMap.V   :HeatMap   [T,Q]   (ratio_M)

In [7]:
# ===== Heatmap 3: Pressure vs Quality =====
print("Calculating P vs Q grid...")

# Create 2D grid
P_range = np.linspace(0.1, 200, N_grid)  # 0.1 to 200 bar
Q_range = np.linspace(0, 1, N_grid)  # Quality from 0 to 1
P_grid, Q_grid = np.meshgrid(P_range, Q_range)

# Flatten for calculation
P_flat = cast(Q[Pressure, Numpy1DArray], Q(P_grid.flatten(), "bar"))
Q_flat = cast(Q[Dimensionless, Numpy1DArray], Q(Q_grid.flatten(), "dimensionless"))

# Calculate properties for both formulations
w97_pq = Fluid("IF97::Water", P=P_flat, Q=Q_flat)
w95_pq = Fluid("Water", P=P_flat, Q=Q_flat)

# Create dataframe
df_pq = pl.DataFrame(
    {
        "P": P_flat.m,
        "Q": Q_flat.m,
    }
    | {f"w95_{a}": getattr(w95_pq, a).m for a in attrs}
    | {f"w97_{a}": getattr(w97_pq, a).m for a in attrs}
)

# Create heatmaps for each property
plots_pq = [
    df_pq.with_columns((pl.col(f"w95_{a}") / pl.col(f"w97_{a}")).alias(f"ratio_{a}")).hvplot.heatmap(
        x="P",
        y="Q",
        C=f"ratio_{a}",
        title=f"{a}: IAPWS-95/97 ratio (P vs Q)",
        xlabel="Pressure (bar)",
        ylabel="Quality (vapor fraction)",
        cmap="RdBu_r",
        clim=(0.95, 1.05),
        colorbar=True,
        width=500,
        height=400,
    )
    for a in attrs
]

print("P vs Q plots ready")
sum(plots_pq[1:], plots_pq[0]).cols(2)

Calculating P vs Q grid...
P vs Q plots ready


:Layout
   .HeatMap.I   :HeatMap   [P,Q]   (ratio_D)
   .HeatMap.II  :HeatMap   [P,Q]   (ratio_H)
   .HeatMap.III :HeatMap   [P,Q]   (ratio_S)
   .HeatMap.IV  :HeatMap   [P,Q]   (ratio_V)
   .HeatMap.V   :HeatMap   [P,Q]   (ratio_M)

## IAPWS-95 vs IAPWS-97: Key Differences

### IAPWS-95 (Scientific Standard)
- **Type**: Fundamental equation of state based on Helmholtz free energy
- **Accuracy**: Highest accuracy available (±0.001% for most properties in the range 0-1000°C, 0-1000 MPa)
- **Coverage**: Entire fluid region from triple point to 1000°C and 1000 MPa
- **Computation**: More complex calculations, slower performance
- **Use case**: Scientific research, high-precision applications, reference standard

### IAPWS-97 (Industrial Formulation)
- **Type**: Simplified practical formulation with piecewise equations for different regions
- **Accuracy**: Industrial accuracy (typically ±0.1-2% depending on property and region)
- **Coverage**: Industrial range up to 800°C and 100 MPa (with extensions to 2000°C)
- **Computation**: Optimized for speed with simpler polynomial equations
- **Use case**: Power plant design, process engineering, real-time simulations

### Key Trade-offs
1. **Speed vs Accuracy**: IAPWS-97 is 5-50x faster but less accurate
2. **Consistency**: IAPWS-95 is thermodynamically consistent everywhere; IAPWS-97 has small discontinuities at region boundaries
3. **Engineering Practice**: For most industrial applications where measurement uncertainty is 1-2% or higher, IAPWS-97 is perfectly adequate and preferred for computational efficiency

## Speed Benchmarks

Let's measure the actual performance difference between IAPWS-95 and IAPWS-97 for typical engineering calculations.

In [ ]:
def benchmark_fluid(backend: str, n_points: int = 1000) -> dict[str, float]:
    """Benchmark fluid property calculations."""

    # Test case 1: Single-phase liquid (common in process engineering)
    T_test = cast(Q[Temperature, Numpy1DArray], np.linspace(Q(20, "degC"), Q(300, "degC"), n_points))
    P_test = Q(10, "bar")

    start = time.perf_counter()
    fluid = Fluid(backend, T=T_test, P=P_test)
    init_time = time.perf_counter() - start

    # Measure property access time
    start = time.perf_counter()
    _ = fluid.D
    _ = fluid.H
    _ = fluid.S
    _ = fluid.V
    _ = fluid.M
    property_time = time.perf_counter() - start

    total_time = init_time + property_time

    return {
        "init_time": init_time,
        "property_time": property_time,
        "total_time": total_time,
        "points": n_points,
    }


# Run benchmarks
print("Running benchmarks with 1000 points...")
results_97 = benchmark_fluid("IF97::Water", 1000)
results_95 = benchmark_fluid("Water", 1000)

print("\n" + "=" * 60)
print("BENCHMARK RESULTS")
print("=" * 60)
print("\nIAPWS-97:")
print(f"  Initialization: {results_97['init_time'] * 1000:.2f} ms")
print(f"  Property access: {results_97['property_time'] * 1000:.2f} ms")
print(f"  Total: {results_97['total_time'] * 1000:.2f} ms")

print("\nIAPWS-95:")
print(f"  Initialization: {results_95['init_time'] * 1000:.2f} ms")
print(f"  Property access: {results_95['property_time'] * 1000:.2f} ms")
print(f"  Total: {results_95['total_time'] * 1000:.2f} ms")

speedup = results_95["total_time"] / results_97["total_time"]
print(f"\nSpeedup: IAPWS-97 is {speedup:.1f}x faster than IAPWS-95")
print(f"Time saved per 1000 calculations: {(results_95['total_time'] - results_97['total_time']) * 1000:.2f} ms")
print("=" * 60)

Running benchmarks with 1000 points...

BENCHMARK RESULTS

IAPWS-97:
  Initialization: 0.01 ms
  Property access: 3.21 ms
  Total: 3.21 ms

IAPWS-95:
  Initialization: 0.00 ms
  Property access: 21.94 ms
  Total: 21.94 ms

Speedup: IAPWS-97 is 6.8x faster than IAPWS-95
Time saved per 1000 calculations: 18.73 ms


In [ ]:
# Test two-phase calculations (common in steam systems)
print("Testing two-phase (saturated steam) calculations...")

Q_test = cast(Q[Dimensionless, Numpy1DArray], np.linspace(Q(0), Q(1), 1000))
P_sat = Q(10, "bar")

start = time.perf_counter()
fluid_97_sat = Fluid("IF97::Water", Q=Q_test, P=P_sat)
_ = fluid_97_sat.D, fluid_97_sat.H, fluid_97_sat.S
time_97_sat = time.perf_counter() - start

start = time.perf_counter()
fluid_95_sat = Fluid("Water", Q=Q_test, P=P_sat)
_ = fluid_95_sat.D, fluid_95_sat.H, fluid_95_sat.S
time_95_sat = time.perf_counter() - start

print("\nTwo-phase calculations (1000 points):")
print(f"  IAPWS-97: {time_97_sat * 1000:.2f} ms")
print(f"  IAPWS-95: {time_95_sat * 1000:.2f} ms")
print(f"  Speedup: {time_95_sat / time_97_sat:.1f}x")

Testing two-phase (saturated steam) calculations...

Two-phase calculations (1000 points):
  IAPWS-97: 3.25 ms
  IAPWS-95: 6.53 ms
  Speedup: 2.0x
